In [ ]:
# !unzip data/election-day-tweets.zip -d data/

In [31]:
!pip install whoosh
!pip install gensim
!pip install sumy
!pip install textblob
!pip install flask

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import unicodedata
import gensim
import nltk
import html
import json
import os

from IPython.display import display, HTML
from whoosh.fields import Schema, ID, KEYWORD, TEXT
from whoosh.index import create_in
from whoosh.query import Term
from IPython.display import display, HTML
from tqdm import tqdm

pd.set_option('display.max_colwidth', 500)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
tqdm.pandas()

[nltk_data] Downloading package punkt to /Users/nellex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nellex/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/nellex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
tweet_para_df = pd.read_csv('data/election-day-tweets.csv', sep=',', header=0)
tweet_para_df = tweet_para_df.sample(frac=0.1, random_state=42)

tweet_para_df = tweet_para_df[['id', 'text']]
tweet_para_df['tweet_text'] = tweet_para_df.apply(
    lambda row: 
    str(unicodedata.normalize('NFD', html.unescape(row['text'].replace('\n', ' ')))
        .encode('ascii', 'ignore'), 
        'utf-8'), 
    axis=1)
tweet_para_df.drop(['text'], inplace=True, axis=1)
display(tweet_para_df)

,id,tweet_text
136241,796198280476708864,Any regrets? https://t.co/5m1TwFOpd7
62702,796234506365640704,"House & Senate will hold as GOP controlled, SC will be conservative for decades... I wouldn't be too certain https://t.co/VT6XFJ0IBY"
366061,796014295184093189,#Election2016 fingers crossed.... https://t.co/BCMnxcHIMe
288852,795988106625290240,"One more day until people go back to normal, and the office stops talking about politics for 4 years! #Election2016"
248613,795971202321944577,In line...at the polls...at 6:43 am! Let's do this! #ElectionDay #election2016 #HillaryClinton #trump https://t.co/1jzFZGEz5F
212440,795937294448553984,Top Trends Sweden-Nov08 11:19 CET Sanna Nielsen #Election2016 Ilmar Reepalu #ImWithHer Stevie Wonder https://t.co/75j1lqqN7Y
88893,796325001288908800,Its funny how people are getting so salt over trump winning even though the president doesnt make a lot of the choices congress does
109644,796068056346886144,"Look I always vote democrat. But if Trump was running democrat, I'd vote for a republican. That's how full of nonsense this bs is"
51077,796215704420749315,"@dccommonsense @bhtru yes, because a president Trump with SCOTUS, and a unified Congress will just be ITCHING to reform elections"
263609,795978906486669312,RT RelatabIetxts: Who are you going to vote? #ElectionDay #Election2016 #Vote2016 #USADecides (Pls retweet after voting)


In [30]:
# Index for Election Tweets

ELECTION_TWEET_INDEX_DIR = "election_tweet_index"

schema = Schema(content=TEXT, idx=ID(unique=True, stored=True))
 
# Create index dir if it does not exists.
if not os.path.exists(ELECTION_TWEET_INDEX_DIR):
    os.mkdir(ELECTION_TWEET_INDEX_DIR)

# Initialize index
search_index = create_in(ELECTION_TWEET_INDEX_DIR, schema)

# Fill index with posts from DB
writer = search_index.writer()

for sqIdx, row in tqdm(tweet_para_df.iterrows()):
    writer.update_document(content=row['tweet_text'], idx=str(row['id']))

writer.commit()

39763it [00:15, 2595.92it/s]


In [4]:
# Load the pre-generated index instead

from whoosh.fields import Schema, ID, KEYWORD, TEXT
from whoosh.index import create_in, open_dir
from whoosh.query import Term
from IPython.display import display, HTML
from tqdm import tqdm

ELECTION_TWEET_INDEX_DIR = "election_tweet_index"

search_index = open_dir(ELECTION_TWEET_INDEX_DIR)

In [5]:
tags_to_process = ('JJ', 'NN')

def fetch_intent_from_question(text, tags=tags_to_process):
    tokenized = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokenized)
    entities = [word for word, tag in tagged if tag.startswith(tags)]
    if not entities:
        return tokenized
    return entities

fetch_intent_from_question("Will Hillary stand by her promise?")

['Hillary', 'promise']

In [6]:
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh import qparser

# Search inside index for post containing "test", then it displays
# results.
searcher = search_index.searcher()
og = qparser.OrGroup.factory(0.95)
qp = QueryParser("content", schema=search_index.schema, group=og)


def find_candidate_answer_set(Q):
    intent = fetch_intent_from_question(Q)
    query = qp.parse(" ".join(intent))

    result = searcher.search(query, limit=100)    
    result_index_list = []
    for r in result: result_index_list.append(int(r['idx']))
    
    return result_index_list

Q = "Donald Trump will crush Hillary?"
result_index_list = find_candidate_answer_set(Q)
display(tweet_para_df[tweet_para_df.id.isin(result_index_list)])

,id,tweet_text
246422,795968823572754432,Prediction for #election2016 is either a comfortable Hillary Clinton win or a marginal Donald Trump win - hopefully I can get a call right!
225797,795949870653771776,Donald Trump is everything that's wrong with our culture. Hillary Clinton is everything that's wrong with our government. #Election2016
189172,795872814486142976,Donald Trump or Hillary Clinton? #Election2016
122681,796153703145291776,Republican Donald Trump and Democrat Hillary Clinton have scored early victories in their bitter presidential... https://t.co/yyi0EthGCw
156198,796266957263818752,#Elections2016 : Republican Donald Trump wins 288 electoral colleges against Democrat Hillary Clinton's 215 to emer https://t.co/r9WA0niWFW
264563,795979274696200192,"#Election2016 my gut instinct says donald trump will win,,, even though hillary has a more chance of winning"
307873,795994539483918336,Plot twist: Donald trump is voting for Hillary #Election2016
246807,795969255959302145,I'm so worried for you guys & I'm not even American. Vote wisely. Vote for Hillary. Don't let Donald Trump ruins America. #Election2016
231518,795953873273884672,Imagine being American right now and having to choose either Donald Trump or Hillary Clinton to run your country #Election2016
321340,795999053251280896,Half Naked Women protesting against Donald Trump in polling area. These are the nuts that support Hillary Clinton. https://t.co/YkoO4BCy5d


In [7]:
# Load Google's pre-trained Word2Vec model.

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', 
                                                            binary=True)

In [8]:
import itertools
import nltk
import numpy as np

def feature_alignment_score(row):
    global counter_value
    intents = [x for x in fetch_intent_from_question(row['question']) if x in w2v_model.vocab]
    tokens = [x for x in nltk.word_tokenize(row['sentence']) if x in w2v_model.vocab]
    total_items = len(intents) * len(tokens)
    iter_items = itertools.product(intents, tokens)
    alignment_score = 0.0
    for a, b in iter_items:
        alignment_score += w2v_model.distance(a, b)
    if total_items == 0: return float('NaN')
    return alignment_score / total_items

In [9]:
def feature_max_alignment(row):
    intents = [x for x in fetch_intent_from_question(row['question']) if x in w2v_model.vocab]
    tokens =  [x for x in fetch_intent_from_question(row['sentence']) if x in w2v_model.vocab]
    if len(intents) * len(tokens) == 0: return float('NaN')
    max_alignment_score = 0.0
    for intent in intents:
        max_alignment_score += np.min([w2v_model.distance(x, intent) for x in tokens])
    return max_alignment_score / len(intents)

In [10]:
def feature_number_of_intent(row):
    intents = [x for x in fetch_intent_from_question(row['question']) if x in w2v_model.vocab]
    return len(intents)

In [11]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [12]:
from collections import Counter
from nltk.stem import PorterStemmer

def feature_exact_match(row):
    ps = PorterStemmer()
    return sum(
        (
            Counter([ps.stem(x) for x in row['sentence'].lower().split()]) 
            & 
            Counter([ps.stem(x) for x in row['question'].lower().split()])
        ).values()
    )

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

def feature_count_vectorizer(feature_df):
    ctv = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word',token_pattern=r'\w{1,}', 
                          ngram_range=(1, 2), stop_words = 'english')
    ctv.fit(feature_df.sentence.values + feature_df.question.values)

    sent_ctv = ctv.transform(feature_df.sentence.values)
    ques_ctv = ctv.transform(feature_df.question.values)

    ctv_score = []
    for s, q in tqdm(zip(sent_ctv, ques_ctv)):
        dot = s.dot(q.T).data
        val = dot[0] if len(dot) else 0
        ctv_score.append(val)

    return pd.Series(ctv_score).values

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def feature_tfidf(feature_df):
    tfv = TfidfVectorizer(tokenizer=LemmaTokenizer(), min_df=3,  max_features=None, 
                strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
                stop_words = 'english')
    tfv.fit(feature_df.sentence.values)

    sent_tfv = tfv.transform(feature_df.sentence.values)
    ques_tfv = tfv.transform(feature_df.question.values)

    tfv_score = []
    for s, q in tqdm(zip(sent_tfv, ques_tfv)):
        dot = s.dot(q.T).data
        val = dot[0] if len(dot) else 0.0
        tfv_score.append(val)
        
    return pd.Series(tfv_score).values

In [15]:
def generate_feature(Q):
    result_index_list = find_candidate_answer_set(Q)
    feature_df = tweet_para_df[tweet_para_df.id.isin(result_index_list)].copy()
    
    feature_df.rename(columns={'tweet_text': 'sentence'}, inplace=True)
    feature_df['question'] = Q
    
    feature_df['alignment_score'] = feature_df.apply(feature_alignment_score, axis=1)
    feature_df.alignment_score.fillna(-1.0, inplace=True)

    feature_df['max_alignment'] = feature_df.apply(feature_max_alignment, axis=1)
    feature_df.max_alignment.fillna(-1.0, inplace=True)

    feature_df['number_of_intent'] = feature_df.apply(feature_number_of_intent, axis=1)
    feature_df['question_words'] = feature_df.question.apply(lambda x: len(x.split()))
    feature_df['exact_match'] = feature_df.apply(feature_exact_match, axis=1)

    feature_df['count_vectorizer_match'] = feature_count_vectorizer(feature_df)
    feature_df['tfidf_match'] = feature_tfidf(feature_df)
    
    feature_df['coverage'] = feature_df.exact_match / feature_df.question_words
    
    return feature_df
    
generate_feature("How is Hillary?")

100it [00:00, 2210.34it/s]
100it [00:00, 2279.98it/s]


,id,sentence,question,alignment_score,max_alignment,number_of_intent,question_words,exact_match,count_vectorizer_match,tfidf_match,coverage
103818,796027043951603712,"@GoFishLynn Sandler now give to CF, & to the Democrat Party & Hillary campaign - more dodgy characters that Hillary & co love to know.",How is Hillary?,0.801519,2.220446e-16,1,3,0,2,0.039670,0.000000
266103,795979846635634689,"Hillary, Hillary, I wish you become the first Lady US President https://t.co/SOcibpazLl",How is Hillary?,0.728329,2.220446e-16,1,3,0,2,0.044751,0.000000
197896,795895103554514944,Hillary better win #Election2016,How is Hillary?,0.662548,2.220446e-16,1,3,0,1,0.061801,0.000000
189172,795872814486142976,Donald Trump or Hillary Clinton? #Election2016,How is Hillary?,0.607569,2.220446e-16,1,3,0,2,0.332669,0.000000
177192,795857044054179840,"Trump or Hillary, I'm gonna miss Obama #Election2016",How is Hillary?,0.713200,2.220446e-16,1,3,0,1,0.033993,0.000000
255012,795975559964725249,"GO HILLARY, GO! #Election2016 @HillaryClinton",How is Hillary?,0.845974,5.595390e-01,1,3,0,1,0.034375,0.000000
266227,795979890935889920,#Election2016 good luck (hillary must win),How is Hillary?,0.827696,4.789451e-01,1,3,0,1,0.045636,0.000000
307873,795994539483918336,Plot twist: Donald trump is voting for Hillary #Election2016,How is Hillary?,0.811614,2.220446e-16,1,3,1,1,0.029126,0.333333
262092,795978335851520001,#NastyWomen4Hillary #iWillVote! #HillaryClinton #NastyWomen #debate #election2016 #Hillary,How is Hillary?,0.762607,2.220446e-16,1,3,0,1,0.034698,0.000000
233568,795955237890695168,Great!!! It's over!!!. Hillary won!!! https://t.co/tcvNw7ds0q,How is Hillary?,0.776320,2.220446e-16,1,3,0,1,0.019479,0.000000


In [26]:
# Load classifier model

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

def fetch_resuts_for_question(Q):
    filename = 'model/random_forest_model_sample.sav'
    model = joblib.load(filename)

    feature_df = generate_feature(Q)
    features = feature_df[['question_words', 'exact_match', 'coverage', 'number_of_intent', \
                           'alignment_score', 'max_alignment', 'count_vectorizer_match', 'tfidf_match']]
    predicted = model.predict_proba(features)[:,1]
    feature_df['score'] = pd.Series(predicted).values
    feature_df.sort_values(by='score', ascending=False, inplace=True)

    return (feature_df)

fetch_resuts_for_question("How is Hillary?")

100it [00:00, 2551.47it/s]
100it [00:00, 2535.89it/s]


,id,sentence,question,alignment_score,max_alignment,number_of_intent,question_words,exact_match,count_vectorizer_match,tfidf_match,coverage,score
348324,796007990843363328,"So, Hillary is Batman? https://t.co/bv6kZ1vzfj",How is Hillary?,0.714419,2.220446e-16,1,3,1,2,0.493514,0.333333,0.809667
8584,795986326495645698,@CBSNews If Hillary wins-Can Congress stop the scandalous nature of Hillary?Can they protect us from being her prey? Vote Trump!,How is Hillary?,0.786155,2.220446e-16,1,3,0,5,0.656810,0.000000,0.793776
197652,795894701652148224,Will anti-Trump immigrant wave prove strong enough for Hillary? #election2016 #hillary #trump #usa #usele https://t.co/3GnwXYzgdu,How is Hillary?,0.814100,2.220446e-16,1,3,1,4,0.352648,0.333333,0.789133
134171,796191862562308097,#3Novices : EXCLUSIVE: Hillary wins Florida? Republican chief reveals Democrats win in vital county https://t.co/svskluqjYo HILLARY Clint,How is Hillary?,0.813238,2.220446e-16,1,3,0,3,0.347694,0.000000,0.700904
159315,796297527805046785,Is the democratic woman Hillary the democratic leader still for the democrat party so can Hillary covern the democratic states ? RT,How is Hillary?,0.805727,2.220446e-16,1,3,1,3,0.776727,0.333333,0.676888
189172,795872814486142976,Donald Trump or Hillary Clinton? #Election2016,How is Hillary?,0.607569,2.220446e-16,1,3,0,2,0.332669,0.000000,0.471353
212454,795937309233442816,Hillary or Trump ? #Election2016,How is Hillary?,0.612044,2.220446e-16,1,3,0,2,0.540917,0.000000,0.433865
386889,796030312035676160,Hillary is like the Nickelback of politicians #Election2016,How is Hillary?,0.772657,2.220446e-16,1,3,1,1,0.109528,0.333333,0.431349
6552,795968415320129536,Hillary is a criminal https://t.co/62CXgzwpj9,How is Hillary?,0.695364,2.220446e-16,1,3,1,1,0.070790,0.333333,0.428553
169068,795846761692528640,Related...we are all voting for #hillary right? #Election2016,How is Hillary?,0.864070,4.789451e-01,1,3,0,2,0.463318,0.000000,0.421538


In [27]:
from sumy.parsers.plaintext import PlaintextParser 
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.nlp.stemmers import Stemmer
from nltk.corpus import stopwords
import re

def summarize_results(Q, feature_df):
    feature_df = feature_df[~feature_df.sentence.str.contains("\?")]
    text = ". ".join(feature_df[feature_df.score > 0.4]
                     .head(5).sentence.tolist())
    text = re.sub(r"([#@])(\w+)\b", '', text)
    text = re.sub(r'http([a-zA-Z0-9/:\.]+)\b', '', text)
    parser = PlaintextParser(text, Tokenizer("english"))
    
    # print ("Text:", text)
    
    summarizer = EdmundsonSummarizer(Stemmer('english'))
    summarizer.stop_words = stopwords.words('english')
    summarizer.bonus_words = fetch_intent_from_question(text, ('JJ')) + nltk.word_tokenize(Q)
    summarizer.stigma_words = ['?', 'when', 'how', 'who'] # stopwords.words('english') # nltk.word_tokenize(Q)
    summarizer.null_words = stopwords.words('english')


    return " ".join([str(s) for s in summarizer(parser.document, 3)])
    
# summarize_results(fetch_resuts_for_question("Congress will lose this election?"))

In [28]:
from textblob import TextBlob

def fetch_opinion(feature_df):
    text = " ".join(feature_df[feature_df.score > 0.5].sentence.tolist())
    return TextBlob(text).sentiment

In [29]:
def get_answer(Q):
    df = fetch_resuts_for_question(Q)
    response = {}
    response['response'] = summarize_results(Q, df)
    response['polarity'] = fetch_opinion(df).polarity
    response['subjectivity'] = fetch_opinion(df).subjectivity
    return response

get_answer("Did Donald Trump win the US election?")

100it [00:00, 2372.71it/s]
100it [00:00, 2309.79it/s]


{'polarity': 0.2252465277777777,
 'response': "Not only did Donald Trump win but the republicans have kept control of Congress. This is even scarier than some may imagine.. Hillary Clinton and Donald Trump have wrapped up their US Presidential election campaigns - voting opens in a matte . US election: Republican Donald Trump leads over Democrat Hillary Clinton 140-104 in the race to 270 electoral votes.. Donald Trump claims US Election machines 'change votes from Republican to Democrat' as",
 'subjectivity': 0.4613888888888889}

In [30]:
from flask import Flask, request, jsonify

app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.route('/')
def index():
    return "Hello from Chatbot!"

@app.route('/ask')
def ask():
    try:
        Q = request.args.get('q')
        response = jsonify(get_answer(Q))
    except:
        response = jsonify(response="Well I\'m certainly clueless about that :|")
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response
 

if __name__ == '__main__':
    app.run(debug=False, port=8990)

100it [00:00, 2451.52it/s]
100it [00:00, 2471.77it/s]
127.0.0.1 - - [23/Sep/2018 00:13:02] "GET /ask?q=Will%20Donald%20Trump%20win%20the%20elections%3F HTTP/1.1" 200 -
100it [00:00, 2616.50it/s]
100it [00:00, 2602.73it/s]
127.0.0.1 - - [23/Sep/2018 00:13:26] "GET /ask?q=Who%20won%20in%20California%3F HTTP/1.1" 200 -
100it [00:00, 2544.32it/s]
100it [00:00, 2608.19it/s]
127.0.0.1 - - [23/Sep/2018 00:13:55] "GET /ask?q=Are%20people%20voting%20for%20Donald%20Trump%3F HTTP/1.1" 200 -
100it [00:00, 2506.28it/s]
100it [00:00, 2550.75it/s]
127.0.0.1 - - [23/Sep/2018 00:14:13] "GET /ask?q=Is%20anyone%20supporting%20Donald%20Trump%3F HTTP/1.1" 200 -
17it [00:00, 2272.39it/s]
17it [00:00, 2167.80it/s]
127.0.0.1 - - [23/Sep/2018 00:14:51] "GET /ask?q=Who%20is%20winning%20in%20Utah%3F HTTP/1.1" 200 -
100it [00:00, 2474.94it/s]
100it [00:00, 2497.69it/s]
127.0.0.1 - - [23/Sep/2018 00:15:20] "GET /ask?q=Who%20is%20winning%20in%20Texas%3F HTTP/1.1" 200 -
100it [00:00, 2321.21it/s]
100it [00:00, 2472.